In [ ]:
import itk
from itkwidgets import view
from ipywidgets import interactive
import ipywidgets as widgets
import urllib.request

In [ ]:
# Download image
urllib.request.urlretrieve("http://raw.githubusercontent.com/InsightSoftwareConsortium/ITK/master/Examples/Data/VisibleWomanHeadSlice.png", "VisibleWomanHeadSlice.png")

In [ ]:
# Read image
image = itk.imread("VisibleWomanHeadSlice.png")
print("Image Type: %s" % str(itk.template(image)))
print("Image Pixel Type: %s" % str(itk.template(itk.template(image)[1][0])))
view(image)

In [ ]:
# Decompose image in 3 components: R, G, and B
split_images = itk.split_components_image_filter(image)

rgb_viewer = view(image, annotations=False)
viewer = view(split_images[0], cmap="gray_Matlab", annotations=False)

# Define a function to use with ipywidgets `interactive`
def view_channels(index=0):
    # Update the image used in the viewer
    viewer.image = split_images[index]
slider = interactive(view_channels, index=(0, len(split_images)-1, 1))

widgets.HBox([rgb_viewer, widgets.VBox([viewer, slider])])

In [ ]:
print("Blue Image Information: %s" % str(split_images[2]))

In [ ]:
# Recompose image with an Alpha layer.
RGBAImageType = itk.Image[itk.RGBAPixel[itk.UC], image.GetImageDimension()]
compose_image_filter = itk.ComposeImageFilter[split_images[0], RGBAImageType].New()
compose_image_filter.SetInput(0, split_images[0])
compose_image_filter.SetInput(1, split_images[1])
compose_image_filter.SetInput(2, split_images[2])
image_duplicator = itk.ImageDuplicator(split_images[2])
image_duplicator.Update()
alpha_layer = image_duplicator.GetOutput()
alpha_layer.FillBuffer(255)
compose_image_filter.SetInput(3, alpha_layer)

compose_image_filter.Update()
recomposed_image = compose_image_filter.GetOutput()
print(recomposed_image)
view(recomposed_image)